# KLASIFIKASI CNN MNIST 

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D

import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(x_train.shape)
print(x_test.shape)

In [ ]:
plt.figure(figsize=(15, 15))
for i in range(20):
    plt.subplot(5, 4, i + 1)
    plt.imshow(x_test[i], cmap=plt.get_cmap('gray'))

plt.show()

In [ ]:
num_classes = 10
input_shape = (28, 28, 1)

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

print(x_train.shape)

In [ ]:
# Make sure images have shape (28, 28, 1)
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],x_train.shape[2],1))
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],x_train.shape[2],1))

print(x_train.shape)

In [ ]:
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Model

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(16, kernel_size=(3, 3), activation="relu"),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),        
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

### Training

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

### Evaluasi

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(history.epoch, history.history['loss'])
plt.plot(history.epoch, history.history['val_loss'])
plt.title('loss')
plt.legend(['loss', 'val_loss'])

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(history.epoch,history.history['accuracy'])
plt.plot(history.epoch,history.history['val_accuracy'])
plt.title('accuracy')
plt.legend(['accuracy', 'val_accuracy'])

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

predictions = model.predict(x_test)

predictions = np.argmax(predictions, axis=1)
y_test = np.argmax(y_test, axis=1)

print('Confusion Matriks:')
print(confusion_matrix(y_test, predictions))
print()

print('Rlassification Report:')
print(classification_report(y_test, predictions, digits=4))

### Inference

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(x_test[0], cmap=plt.get_cmap('gray'))
plt.show()

In [ ]:
img = x_test[0].reshape(-1, 28, 28, 1)

predict_x = model.predict(img) 
classes_x = np.argmax(predict_x,axis=1)
print(classes_x)

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(x_test[2], cmap=plt.get_cmap('gray'))
plt.show()

In [ ]:
img = x_test[2].reshape(-1, 28, 28, 1)

predict_x = model.predict(img) 
classes_x = np.argmax(predict_x,axis=1)
print(classes_x)